In [153]:
import sys
sys.path.append("C:\\Users\\sefi\\Meta-Modelling\\Sample Size vs Accuracy\\Noise Level 3\\Rastrigin10D\\")
import Utils
import pyDOE
import pandas as pd
import numpy as np
import scipy.stats.distributions as dist
from sklearn import preprocessing
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.preprocessing import MinMaxScaler

## Helper Functions

In [154]:
''' Elastic Net Regression for Minimax Robustness'''
def elastic_net_minimax(train_data,test_data,rat):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:65].values, test_data.values])
    regr=ElasticNet(alpha= 0.009782831659031866 ,random_state=0 , l1_ratio=rat, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:65])) ,  np.array(train_data.iloc[:,66]))
    pred = regr.predict(scaler.transform(test_data))
    return regr,pred

''' Elastic Net Regression for Composite Robustness '''
def elastic_net_composite(train_data,test_data,rat):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:65].values, test_data.values])
    regr=ElasticNet(alpha= 0.006493421766031815 ,random_state=0 , l1_ratio=rat, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:65])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    return regr,pred

## Load Training and Test Data Set initially Generated

In [155]:
train = pd.read_csv(sys.path[1][:-3]+str('Training_Data_Sets\\500Samples.csv')).iloc[:,1:]
for i in range(1,11):
    train ['X'+str(i)+'**2'] = train['X'+str(i)]**2
for i in range(1,11):
    for j in range(i+1,11):
        train ['X'+str(i)+'X'+str(j)] = train['X'+str(i)] * train ['X'+str(j)]
f_original = train['Y = F(X1,X2)']
robust_original = train['Y = robust_regularization(X1,X2)']
composite_original = train ['Y = composite(X1,X2)']
del train['Y = F(X1,X2)']
del train['Y = robust_regularization(X1,X2)']
del train ['Y = composite(X1,X2)']
train['Y = F(X1,X2)'] = f_original
train['Y = robust_regularization(X1,X2)'] = robust_original
train ['Y = composite(X1,X2)'] = composite_original
test = pd.read_csv(sys.path[1][:-3]+str('Test_Data_Sets\\750Samples.csv')).iloc[:,1:]
for i in range(1,11):
    test ['X'+str(i)+'**2'] = test['X'+str(i)]**2
for i in range(1,11):
    for j in range(i+1,11):
        test ['X'+str(i)+'X'+str(j)] = test['X'+str(i)] * test ['X'+str(j)]
true_minimax = np.array(test.True_Minimax)
true_composite = np.array(test.True_Composite)
del test['True_Minimax']
del test ['True_Composite']
test['True_Minimax'] = pd.Series(true_minimax)
test ['True_Composite'] = pd.Series(true_composite)

## ELN RR 

In [156]:
scaler =  MinMaxScaler().fit(np.r_[train.iloc[:,:65].values, test.iloc[:,:65].values])
regr = ElasticNetCV(cv=5,random_state=0)
regr.fit(scaler.transform(np.array(train.iloc[:,:65])) , np.array(train.iloc[:,66]))
print (regr.alpha_)

0.009782831659031866


## ELN RC


In [157]:
scaler =  MinMaxScaler().fit(np.r_[train.iloc[:,:65].values, test.iloc[:,:65].values])
regr = ElasticNetCV(cv=5,random_state=0)
regr.fit(scaler.transform(np.array(train.iloc[:,:65])) , np.array(train.iloc[:,-1]))
print (regr.alpha_)

0.006493421766031815


## ELN

In [158]:
Hyper_Parameters = np.linspace(0,1,1000)
mean_abs_error_minimax = np.zeros(Hyper_Parameters.shape[0])
mean_abs_error_composite = np.zeros(Hyper_Parameters.shape[0])
for i in range(Hyper_Parameters.shape[0]):
    model_minimax,pred_k_m = elastic_net_minimax(train,test.iloc[:,:65],Hyper_Parameters[i])
    model_composite,pred_k_c = elastic_net_composite(train,test.iloc[:,:65],Hyper_Parameters[i])
    mean_abs_error_minimax[i] = np.mean((abs(true_minimax-pred_k_m) / abs(true_minimax) ) * 100)
    mean_abs_error_composite[i] = np.mean((abs( true_composite-pred_k_c) / abs(true_composite) ) * 100)

C:\Users\sefi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [159]:
print ('Best Configuration of Hyper Parameters for Robust Regularization')
Hyper_Parameters[np.argmin(mean_abs_error_minimax)] , np.min(mean_abs_error_minimax)

Best Configuration of Hyper Parameters for Robust Regularization


(0.22322322322322322, 1.7528694847747808)

In [160]:
print ('Best Configuration of Hyper Parameters for Robust Composition')
Hyper_Parameters[np.argmin(mean_abs_error_composite)] , np.min(mean_abs_error_composite)

Best Configuration of Hyper Parameters for Robust Composition


(1.0, 0.7148901226097815)